# Bootcamp FIAP - Turma 11IA

### Integrantes:

###    1) Fabio Rizzi - RM336656
###    2) Jurandir Ventura - RM336634
###    3) Marcelo Preto - RM336632
###    4) Thiago Alexandre – RM336583

## Desafio Bootcamp:

### Analisar o dataset de atendimentos de manutenção da empresa VIVANTE no Hospital Metropolitano e trazer insights e soluções de Machine Learning ou Deep Learning para convergir com o objetivo da empresa em liderar o mercado de Facilities com tecnologias e conhecimento disruptivos.

In [1]:
#Importando as bibliotecas. Se houver algum erro, use "pip install Libname"
import pandas as pd
import seaborn as sb
import numpy as np
import pydotplus
import graphviz
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz, plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
from xgboost import XGBClassifier
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

In [2]:
#Leitura do dataset .csv

#Lembrar que a origem da dataset é uma variável.
#Neste caso é proveniente da pasta de datasets do Github: https://github.com/fiapIA/bootcamp/dataset
dataset = pd.read_excel('dataset/VIVANTE_DATASET_PIVOTADO_V1.xlsx')


#Visualização das 5 primeiras linhas do dataset
dataset.head(5)

,Número OS,Denominação OS,Ativo,Descrição Ativo,Estado OS,Denominação Estado OS,Classe de Trabalho,Descrição Classe de Trabalho,Atividade,Descrição Atividade,...,Previsão 1º Atend.,Roteiro,Descrição Roteiro,Data Início Trabalho,Data Final Trabalho,Tempo Trabalho Feedback M.Obra,Tempo Trabalho Convertido,Técnico,nome_tecn,Tempo Previsto
0,0,DDS (DIÁLOGO DIÁRIO DE SEGURANÇA),HMBHPA1SSSRE03,SALA DE REUNIÃO VIVANTE,FC,Fechada,TRT,TREINAMENTO,TRT,TREINAMENTOS,...,2020-04-23 19:30:00,NaN,NaN,2020-04-23 19:31:00,2020-04-23 20:00:00,0.483333,00:29:00,VB221378,LEONARDO DO NASCIMENTO COSTA,3.0
1,204472,TREINAMENTO SEMESTRAL (QGBT'S - OPERAÇÃO E MAN...,HMEB-ELE.T004,TREINAMENTO SEMESTRAL (QGBT'S - OPERAÇÃO E MAN...,FC,Fechada,PRS,PREVENTIVA SISTEMÁTICA,TRT,TREINAMENTOS,...,2019-07-01 06:00:00,ELE.T004.06,TREINAMENTO SEMESTRAL (QGBT'S - OPERAÇÃO E MAN...,2020-01-01 06:00:00,2020-01-01 06:01:00,0.016667,00:01:00,VB225518,LUCAS LIMA RAMOS,NaN
2,215523,PIA VAZANDO,HMBHPA1PVWC014,SANITÁRIO FEMININO SALA DE AULA 01,FC,Fechada,COR,CORRETIVA EMERGÊNCIAL,HIDR,HIDRAULICA,...,2020-01-22 14:16:00,NaN,NaN,2020-01-22 13:52:00,2020-01-22 14:07:00,0.250000,00:15:00,vb104104,JOSELUCIO ROSA DE ANDRADE,3.0
3,215749,VERIFICAR EQUIPAMENTO,HMBHPA3PVCME01,ARSENAL CME,FC,Fechada,COR,CORRETIVA EMERGÊNCIAL,ENCL,ENGENHARIA CLÍNICA,...,2020-01-25 12:37:00,NaN,NaN,2020-01-25 10:46:00,2020-01-25 11:30:00,0.733333,00:44:00,VB220999,IVOMAR ROSSI CRUZ,3.0
4,247489,INTERVENÇAO CRITICA ENGENHARIA CLINICA ANUAL C...,1E01,ESFIGMOMANOMETRO DE PEDESTAL - 608902-N/A,FC,Fechada,PRS,PREVENTIVA SISTEMÁTICA,ENCL,ENGENHARIA CLÍNICA,...,2020-01-01 06:00:00,ICAANCALIB,INTERVENÇAO CRITICA ENGENHARIA CLINICA ANUAL C...,2020-01-06 14:06:00,2020-01-06 14:30:00,0.400000,00:24:00,VB217522,LUDMILA KAREN OLIVEIRA DE PAULA,4.0


In [3]:
# Analisando o tamanho do dataset
dataset.shape

(34106, 23)

In [4]:
# Visão geral dos dados (com DataType)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34106 entries, 0 to 34105
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Número OS                       34106 non-null  int64         
 1   Denominação OS                  34106 non-null  object        
 2   Ativo                           34106 non-null  object        
 3   Descrição Ativo                 34106 non-null  object        
 4   Estado OS                       34106 non-null  object        
 5   Denominação Estado OS           34106 non-null  object        
 6   Classe de Trabalho              34106 non-null  object        
 7   Descrição Classe de Trabalho    34106 non-null  object        
 8   Atividade                       34106 non-null  object        
 9   Descrição Atividade             34106 non-null  object        
 10  Data da Criação                 34106 non-null  datetime64[ns]
 11  Da

## É possível verificar que temos várias colunas do tipo 'object'. Algumas, de acordo com o dicionário disponibilizado são categóricos. Vamos analisar o nome_tecn (Nome do Técnico) e a Descrição Classe de Trabalho.

In [5]:
#Agrupando os valores dessa coluna, para entender se são categóricos.

dataset.groupby(['nome_tecn'])['nome_tecn'].count()

nome_tecn
ADENILSON MOREIRA VAZ                      438
AILSON LIMA DOS SANTOS                     195
ALEXANDRE ANTONIO DOS REIS                 294
ALEXSANDRO DA COSTA                       1613
ANDERSON NASÁRO DA SILVA                    56
ANTONIO LOPES DA SILVA                     462
ANTONIO LUIZ DA ROCHA                     1069
AVELINO PEREIRA DE MORAIS                  172
CAIO CESAR WENCESLAU                       760
CARLOS ANTONIO                             740
CEZAR LUCAS SILVA DE OLIVEIRA             1199
DAVID BRENO BRAGA                          400
DENER SILVEIRA RODRIGUES                   324
EDER EDUARDO FERREIRA                      654
EDINALDO VIANA                             164
ELIZEU DOS SANTOS JUNIOR                   835
EULER ANDRADE                              649
EXPEDITO SILVA CAMPOS JUNIOR              1070
FILIPE HENRIQUE DOS SANTOS                 145
GILSON CLAUDIO DA SILVA                    735
HUGO LEONARDO DE SANTANA                   425
ISA

In [6]:
dataset.groupby(['Descrição Classe de Trabalho'])['Descrição Classe de Trabalho'].count()

Descrição Classe de Trabalho
ATENDIMENTO DE SOLICITAÇÕES      531
CHAMADO                            6
CORRETIVA EMERGÊNCIAL           9707
CORRETIVA PROGRAMADA            2306
INSTALAÇÃO                         1
OBRA                              83
PREVENTIVA PROATIVA                8
PREVENTIVA SISTEMÁTICA         12789
ROTINA DE INSPEÇÃO              6178
TREINAMENTO                     2497
Name: Descrição Classe de Trabalho, dtype: int64

In [7]:
# Verificação de dados nulos
dataset.isnull().sum()

Número OS                             0
Denominação OS                        0
Ativo                                 0
Descrição Ativo                       0
Estado OS                             0
Denominação Estado OS                 0
Classe de Trabalho                    0
Descrição Classe de Trabalho          0
Atividade                             0
Descrição Atividade                   0
Data da Criação                       0
Data/Hora Abertura                    0
Data/Hora de Conclusão              520
Previsão 1º Atend.                    7
Roteiro                           15410
Descrição Roteiro                 15410
Data Início Trabalho                490
Data Final Trabalho                 490
Tempo Trabalho Feedback M.Obra      490
Tempo Trabalho Convertido             0
Técnico                             490
nome_tecn                          1135
Tempo Previsto                      817
dtype: int64

### Existem muitos Roteiros de Trabalho em branco, com concentração em Corretivas Emergenciais, então precisamos entender se uma melhoria pode estar voltada para essa classe de trabalho. O que podemos fazer é excluir essa coluna da análise.

In [8]:
# Algumas colunas são redundantes, pois trazem a descrição e o código, então vamos retirar o que é dicionário
#Usando a descrição fica mais fácil de entender

# Removendo colunas que não precisamos usar:


try:
    dataset.drop(columns=['Número OS','Denominação OS','Estado OS',
                          'Classe de Trabalho', 'Atividade', 'Previsão 1º Atend.',
                          'Roteiro', 'Descrição Roteiro', 'Técnico'], inplace=True)
except:
    print('Colunas já removidas')
        

In [9]:
# Verificação de dados nulos
dataset.isnull().sum()

Ativo                                0
Descrição Ativo                      0
Denominação Estado OS                0
Descrição Classe de Trabalho         0
Descrição Atividade                  0
Data da Criação                      0
Data/Hora Abertura                   0
Data/Hora de Conclusão             520
Data Início Trabalho               490
Data Final Trabalho                490
Tempo Trabalho Feedback M.Obra     490
Tempo Trabalho Convertido            0
nome_tecn                         1135
Tempo Previsto                     817
dtype: int64

In [10]:
# Removendo dados nulos

dataset.dropna(inplace=True)

In [11]:
# Verificação de dados nulos novamente, para certificar
dataset.isnull().sum()

Ativo                             0
Descrição Ativo                   0
Denominação Estado OS             0
Descrição Classe de Trabalho      0
Descrição Atividade               0
Data da Criação                   0
Data/Hora Abertura                0
Data/Hora de Conclusão            0
Data Início Trabalho              0
Data Final Trabalho               0
Tempo Trabalho Feedback M.Obra    0
Tempo Trabalho Convertido         0
nome_tecn                         0
Tempo Previsto                    0
dtype: int64

In [12]:
# Visão geral dos dados (com DataType) novamente

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32748 entries, 0 to 34104
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Ativo                           32748 non-null  object        
 1   Descrição Ativo                 32748 non-null  object        
 2   Denominação Estado OS           32748 non-null  object        
 3   Descrição Classe de Trabalho    32748 non-null  object        
 4   Descrição Atividade             32748 non-null  object        
 5   Data da Criação                 32748 non-null  datetime64[ns]
 6   Data/Hora Abertura              32748 non-null  datetime64[ns]
 7   Data/Hora de Conclusão          32748 non-null  datetime64[ns]
 8   Data Início Trabalho            32748 non-null  datetime64[ns]
 9   Data Final Trabalho             32748 non-null  datetime64[ns]
 10  Tempo Trabalho Feedback M.Obra  32748 non-null  float64       
 11  Te

In [13]:
# Vamos renomear algumas colunas para ficar mais fácil trabalhar no pandas

try:
    dataset.rename(columns = {'Descrição Ativo': 'Desc_Ativo',
                          'Denominação Estado OS': 'Estado_OS',
                          'Descrição Classe de Trabalho' : 'Classe_trabalho',
                          'Descrição Atividade' : 'Atividade',
                          'Data da Criação' : 'Data_criada',
                          'Data/Hora Abertura': 'Data_abertura',
                          'Data/Hora de Conclusão' : 'Data_conclusao',
                          'Data Início Trabalho' : 'Data_inicio',
                          'Data Final Trabalho': 'Data_fim',
                          'Tempo Trabalho Feedback M.Obra' : 'Tempo_trab_num',
                          'Tempo Trabalho Convertido': 'Tempo_trab_h',
                          'Tempo Previsto':'Tempo_previsto'}, inplace = True)
    
except:
    print('Colunas já renomeadas')

In [14]:
# Visão geral dos dados (com DataType) novamente

dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32748 entries, 0 to 34104
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Ativo            32748 non-null  object        
 1   Desc_Ativo       32748 non-null  object        
 2   Estado_OS        32748 non-null  object        
 3   Classe_trabalho  32748 non-null  object        
 4   Atividade        32748 non-null  object        
 5   Data_criada      32748 non-null  datetime64[ns]
 6   Data_abertura    32748 non-null  datetime64[ns]
 7   Data_conclusao   32748 non-null  datetime64[ns]
 8   Data_inicio      32748 non-null  datetime64[ns]
 9   Data_fim         32748 non-null  datetime64[ns]
 10  Tempo_trab_num   32748 non-null  float64       
 11  Tempo_trab_h     32748 non-null  object        
 12  nome_tecn        32748 non-null  object        
 13  Tempo_previsto   32748 non-null  float64       
dtypes: datetime64[ns](5), float64(2), obje

In [15]:
# Analisando a quantidade de atendimentos por ativo - TOP 30

dataset.groupby('Desc_Ativo')['Desc_Ativo'].count().nlargest(30)

Desc_Ativo
SALA DE REUNIÃO VIVANTE         2051
CASA DE BOMBAS                   692
ÁREA COMUM                       666
COZINHA - LAVAGEM DE PANELAS     612
CASA DE BOILER                   583
AREA COMUM - ANEXO B             562
TANQUE DE DIESEL                 553
SALA NOBREAK NORTE               552
OFICINA ENGENHARIA CLÍNICA       472
CABINE DE TRANSFORMAÇÃO          432
TAN-O²                           298
FARMACIA ESTOQUE BRUTO           265
UDC                              235
MONITOR MULTIPARÂMETRO           234
AVC                              209
AREA COMUM - 1°SS                188
CTI 6                            188
VENTILADOR PULMONAR              182
SALA DE EQUIPAMENTOS CTI 4       176
MONITOR MULTIPARAMETRO           164
BLOCO CIRURGICO CORREDOR         163
5º ALA SUL                       162
CTI 5                            160
AREA COMUM - TERREO              154
AREA COMUM - PILOTS              148
AREA COMUM - 3°SS                145
SALA DE EQUIPAMENTOS CTI 1 

In [16]:
# Analisando quantos ativos temos no dataset

#dataset.loc[dataset['Ativo']].agg(['nunique','count','size'])

num_ativos = dataset['Ativo'].nunique()

print ('Temos' , num_ativos, 'ativos monitorados ou para realizar manutenção no Dataset.')

Temos 3373 ativos monitorados ou para realizar manutenção no Dataset.


In [17]:
# Analisando a somatória de tempo de manutenção por Ativo - TOP 30

dataset.groupby('Desc_Ativo')['Tempo_trab_num'].sum().nlargest(30)

#dataset.groupby('Ativo')['Tempo_trab_num'].sum().nlargest(20)

Desc_Ativo
SALA DE REUNIÃO VIVANTE                   1117.000000
JARDIM                                     861.783333
ÁREA COMUM                                 536.833333
TAN-O²                                     411.983333
OFICINA ENGENHARIA CLÍNICA                 381.450000
CASA DE BOMBAS                             364.433333
CASA DE BOILER                             315.716667
AREA COMUM - 1°SS                          297.100000
AREA COMUM - 3°SS                          242.216667
AREA COMUM - PILOTS                        236.250000
SALA NOBREAK NORTE                         213.533333
AREA COMUM - ANEXO B                       212.833333
TANQUE DE DIESEL                           194.166667
UDC                                        183.450000
COZINHA - LAVAGEM DE PANELAS               182.600000
CABINE DE TRANSFORMAÇÃO                    170.683333
FARMACIA ESTOQUE BRUTO                     169.783333
VESTIARIO FEMININO ACESSO FUNCIONÁRIOS     148.233333
MONITOR MULTIPARÂ

In [18]:
# Analisando a quantidade de atendimentos por ativo - TOP 30

dataset.groupby('Classe_trabalho')['Classe_trabalho'].count().nlargest(30)

Classe_trabalho
PREVENTIVA SISTEMÁTICA         12780
CORRETIVA EMERGÊNCIAL           9577
ROTINA DE INSPEÇÃO              6129
TREINAMENTO                     2484
CORRETIVA PROGRAMADA            1680
OBRA                              83
PREVENTIVA PROATIVA                8
CHAMADO                            4
ATENDIMENTO DE SOLICITAÇÕES        2
INSTALAÇÃO                         1
Name: Classe_trabalho, dtype: int64

In [19]:
# Analisando a somatória de tempo de manutenção por Classe de Trabalho - TOP 30

dataset.groupby('Classe_trabalho')['Tempo_trab_num'].sum().nlargest(30)

Classe_trabalho
PREVENTIVA SISTEMÁTICA         7654.900000
CORRETIVA EMERGÊNCIAL          6980.366667
ROTINA DE INSPEÇÃO             3254.550000
CORRETIVA PROGRAMADA           2524.766667
TREINAMENTO                    1363.950000
OBRA                            197.250000
PREVENTIVA PROATIVA              10.216667
ATENDIMENTO DE SOLICITAÇÕES       0.850000
INSTALAÇÃO                        0.816667
CHAMADO                           0.466667
Name: Tempo_trab_num, dtype: float64

## Convertendo Dados Categóricos em Numéricos

In [41]:
#Agora convertendo a classe de trabalho em números.

dataset['Classe_trabalho'] = pd.Categorical(dataset.Classe_trabalho).codes

dataset.head()

,Número OS,Denominação OS,Ativo,Descrição Ativo,Denominação Estado OS,Classe_trabalho,Descrição Atividade,Data da Criação,Data/Hora Abertura,Data/Hora de Conclusão,Previsão 1º Atend.,Data Início Trabalho,Data Final Trabalho,Tempo Trabalho Feedback M.Obra,Tempo Trabalho Convertido,nome_tecn,Tempo Previsto
0,0,DDS (DIÁLOGO DIÁRIO DE SEGURANÇA),HMBHPA1SSSRE03,SALA DE REUNIÃO VIVANTE,Fechada,9,TREINAMENTOS,2020-04-24 15:19:53.987,2020-04-23 19:00:00,2020-04-23 20:00:00,2020-04-23 19:30:00,2020-04-23 19:31:00,2020-04-23 20:00:00,0.483333,00:29:00,LEONARDO DO NASCIMENTO COSTA,3.0
2,215523,PIA VAZANDO,HMBHPA1PVWC014,SANITÁRIO FEMININO SALA DE AULA 01,Fechada,2,HIDRAULICA,2020-01-22 13:50:40.510,2020-01-22 13:46:00,2020-01-22 14:07:00,2020-01-22 14:16:00,2020-01-22 13:52:00,2020-01-22 14:07:00,0.250000,00:15:00,JOSELUCIO ROSA DE ANDRADE,3.0
3,215749,VERIFICAR EQUIPAMENTO,HMBHPA3PVCME01,ARSENAL CME,Fechada,2,ENGENHARIA CLÍNICA,2020-01-25 09:39:44.573,2020-01-25 09:37:00,2020-01-25 11:30:00,2020-01-25 12:37:00,2020-01-25 10:46:00,2020-01-25 11:30:00,0.733333,00:44:00,IVOMAR ROSSI CRUZ,3.0
4,247489,INTERVENÇAO CRITICA ENGENHARIA CLINICA ANUAL C...,1E01,ESFIGMOMANOMETRO DE PEDESTAL - 608902-N/A,Fechada,7,ENGENHARIA CLÍNICA,2019-12-26 14:01:22.877,2020-01-01 06:00:00,2020-01-06 14:30:00,2020-01-01 06:00:00,2020-01-06 14:06:00,2020-01-06 14:30:00,0.400000,00:24:00,LUDMILA KAREN OLIVEIRA DE PAULA,4.0
5,247490,PREVENTIVA DE ENGENHARIA CLINICA SEMESTRAL - ...,1E01,ESFIGMOMANOMETRO DE PEDESTAL - 608902-N/A,Fechada,7,ENGENHARIA CLÍNICA,2019-12-26 14:01:23.113,2020-01-01 06:00:00,2020-01-12 09:35:00,2020-01-01 06:00:00,2020-01-12 08:51:00,2020-01-12 09:35:00,0.733333,00:44:00,LUDMILA KAREN OLIVEIRA DE PAULA,1.0


# Fazendo a Clusterização

## Modelo K-Means

### Normalizando os dados

In [ ]:
df_normalizacao = StandardScaler().fit_transform(dataset.iloc[:,:8])
df_cluster = StandardScaler().fit_transform(df_normalizacao)

In [ ]:
df_cluster

### Redução de dimensionalidade com PCA

In [ ]:
pca = PCA(n_components=2)

PComponents = pca.fit_transform(df_cluster)

#principalComponents.index = df_cluster.index

maindataset = pd.DataFrame(data = PComponents,
                           columns = ['PComp1', 'PComp2'])

In [ ]:
maindataset.head()

### Aplicação do Método Elbow para identificar a quantidade de clusters a ser utilizado

In [ ]:
inertias =[]
for i in range(1, 30):
    kmeans = KMeans(n_clusters = i, max_iter=500).fit(maindataset)    
    #somatório dos erros quadráticos das instâncias de cada cluster
    inertias.append(kmeans.inertia_)

In [ ]:
# Platando o método Elbow, para analisar o cotovelo e número de clusters

plt.figure(1)
plt.plot(range(1, 30), inertias)
plt.title('Método Elbow')
plt.xlabel('No de clusters')
plt.ylabel('Soma Erro Quadrático')
plt.show()

## Esse método não foi possível analisar de forma direta o cotovelo na curva, indicando um score mais baixo e possivelmente não sendo o melhor modelo.

### Dessa forma, vamos usar o modelo K-Means

In [ ]:
# Clusterização com Kmeans

kmeans = KMeans(n_clusters=5)
kmeans.fit(maindataset)

### Agora incluímos o ID do Cluster no Dataset

In [ ]:
# Inclusão de ID no dataset

maindataset['clusters'] = kmeans.labels_


### Plot dos dados (PCA) e os clusters identificados

In [ ]:
fig, ax = plt.subplots()
plt.scatter(maindataset['PComp1'],maindataset['PComp2'],c=maindataset.clusters, s=80)
plt.scatter(kmeans.cluster_centers_[:,0],kmeans.cluster_centers_[:,1],s=100,c='black',label="Centroids",marker='+')
plt.title("K-Means - 4 clusters")
ax.legend()
ax.grid(True)
plt.show()

In [ ]:
# Inclusão de ID no dataset original, para definição de alvo

dataset['cluster'] = kmeans.labels_

dataset.head()

In [ ]:
dataset.groupby(['cluster'])['cluster'].count()

### Há um cluster que tem volume muito pequeno de dados comparado aos demais (cluster 3)

# Classificação

## Determinação de Features

In [ ]:
X = dataset.iloc[:,0:10]
X

## Determinação da variável Target

In [ ]:
y = dataset['cluster']
y.head(10)

## Definição da base de teste e de treino

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .3, random_state=42)

## Modelo de Árvore de Decisão 

In [ ]:
model_tree = DecisionTreeClassifier(max_depth=7)
model_tree.fit(X_train, y_train)
y_pred_tree = model_tree.predict(X_test)

### Resultado na plotagem de Matriz de Confusão

In [ ]:
plot_confusion_matrix(model_tree, X_test, y_test, normalize = 'true')

print(classification_report(y_test, y_pred_tree))
print(accuracy_score(y_test, y_pred_tree))


## Plot da Árvore de decisão

In [ ]:
import os
os.environ['PATH'].split(os.pathsep)
os.environ['PATH'] += os.pathsep + 'C:\\Program Files (x86)\\Graphviz2.38\\bin'


display(
    graphviz.Source(export_graphviz(
        model_tree, 
        feature_names=dataset.columns.values[0:10],  
        filled=True)))

## Resultado não foi interessante, pois a Árvore é extremamente grande e complexa para analisar

## Modelo de KNN

### Encontrando o valor ideal de k

In [ ]:
k_range=range(1,40)
scores=[]
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_knn = knn.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred_knn))

print(scores)
#Plota os valores de acc. em função do valor escolhido de K
plt.plot(k_range, scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

### Treinando o modelo KNN

In [ ]:
model_KNN = KNeighborsClassifier(n_neighbors=7)

model_KNN.fit(X_train, y_train)

### Testando o modelo criado

In [ ]:
y_pred = model_KNN.predict(X_test)

 ## Uso da Matriz de Confusão, baseado no modelo KNN

In [ ]:
plot_confusion_matrix(model_KNN, X_test, y_test, normalize = 'true')

print(classification_report(y_test, y_pred_knn))
print(accuracy_score(y_test, y_pred_knn))


## Modelo XGBOOST

### Treinando o Modelo XGBOOST

In [ ]:
model_xgboost = XGBClassifier()

model_xgboost.fit(X_train, y_train)

### Testando o modelo XGBOOST

In [ ]:
y_pred_xg = model_xgboost.predict(X_test)

## Resultado do modelo XGBOOST

In [ ]:
plot_confusion_matrix(model_xgboost, X_test, y_test, normalize = 'true')

print(classification_report(y_test, y_pred_xg))
print(accuracy_score(y_test, y_pred_xg))


## Análise Final Clusterização / Classificação:



## Blablá

# Etapa de Regressão, para predição do modelo baseado na coluna V2

In [ ]:
# Retornando ao nosso dataset

dataset.head()

## Determinação de Features

In [ ]:
X_reg = dataset.drop(['V2'], axis=1, inplace=False)

## Determinação de Target

In [ ]:
y_reg = dataset['V2']

## Separação de dados de treino e teste

In [ ]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size= .2, random_state=42)

## Modelo SVR

### O modelo SVR necessita de normalização dos dados. Aplicamos em pipeline.

### Treinando o modelo SVR

In [ ]:
model_SVR = make_pipeline(StandardScaler(), SVR())
model_SVR.fit(X_train_reg, y_train_reg)

### Testando o modelo SVR

In [ ]:
y_pred_svr = model_SVR.predict(X_test_reg)

## Resultado SVR

In [ ]:
mse_SVR = mean_squared_error(y_test_reg, y_pred_svr)
score_svr = model_SVR.score(X_test_reg, y_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_svr)
print("R2:{0:.3f}, MSE:{1:.2f}, RMSE:{2:.2f}"
      .format(score_svr, mse, np.sqrt(mse)))

## Modelo Regressão Linear

### Treinando o modelo de Regressão Linear

In [ ]:
model_Linear = LinearRegression()  
model_Linear.fit(X_train_reg, y_train_reg) 

### Testando o modelo de Regressão Linear

In [ ]:
y_pred_Linear = model_Linear.predict(X_test_reg)

## Resultado Regressão Linear

In [ ]:
mse_Linear = mean_squared_error(y_test_reg, y_pred_Linear)
score_linear = model_Linear.score(X_test_reg, y_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_Linear)
print("Resultado Regressão Linear: R2:{0:.7f}, MSE:{1:.2f}, RMSE:{2:.2f}"
      .format(score_linear, mse, np.sqrt(mse)))

## Modelo ElasticNet

### Treinando o Modelo ElasticNet

In [ ]:
model_elastic = ElasticNet(alpha=0.0001)
model_elastic.fit(X_train_reg, y_train_reg)

### Testando o modelo ElasticNet

In [ ]:
y_pred_elastic = model_elastic.predict(X_test_reg)

## Resultado ElasticNet

In [ ]:

score_elastic = model_elastic.score(X_test_reg, y_test_reg)
mse_elastic = mean_squared_error(y_test_reg, y_pred_elastic)
print("Resultado ElasticNet: R2:{0:.7f}, MSE:{1:.2f}, RMSE:{2:.2f}"
      .format(score_elastic, mse_elastic, np.sqrt(mse_elastic)))

## Modelo Regressão Logística

In [ ]:
model_logistic = LogisticRegression(max_iter = 500)
model_logistic.fit(X_train_reg, y_train_reg)

In [ ]:
y_pred_reglogistic = model_logistic.predict(X_test_reg)

In [ ]:
score_reglogistic = model_logistic.score(X_test_reg, y_test_reg)
mse_reglogistic = mean_squared_error(y_test_reg, y_pred_reglogistic)
print("Resultado Regressão Logística: R2:{0:.7f}, MSE:{1:.2f}, RMSE:{2:.2f}"
      .format(score_reglogistic, mse_reglogistic, np.sqrt(mse_reglogistic)))